# **Original model's Answer Relevancy, Bias, Faithfulness**

In [7]:
pip install -q -U deepeval

Note: you may need to restart the kernel to use updated packages.


In [8]:
%%capture
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [9]:
!pip install -q -U bitsandbytes

In [10]:
!pip install -q --upgrade peft trl

In [11]:
!pip install -q lm-format-enforcer

In [12]:
!pip install -q --upgrade pymilvus openai requests tqdm pandas deepeval

In [13]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
read_key = user_secrets.get_secret("HF_READ")

In [14]:
from huggingface_hub import login
login(token= read_key)

In [15]:
import json
from pydantic import BaseModel
import torch
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import (
    build_transformers_prefix_allowed_tokens_fn,
)
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

from deepeval.models import DeepEvalBaseLLM


class CustomModel(DeepEvalBaseLLM):
    def __init__(self):
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )

        model_4bit = AutoModelForCausalLM.from_pretrained(
            "unsloth/gemma-2b-it-bnb-4bit",
            device_map="auto",
            quantization_config=quantization_config,
        )
        tokenizer = AutoTokenizer.from_pretrained(
            "unsloth/gemma-2b-it-bnb-4bit"
        )

        self.model = model_4bit
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        model = self.load_model()
        # Initialize the pipeline for text generation
        gen_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_length=2500,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )

        # Create parser required for JSON confinement using lmformatenforcer
        parser = JsonSchemaParser(schema.schema())
        prefix_function = build_transformers_prefix_allowed_tokens_fn(
            gen_pipeline.tokenizer, parser
        )

        # Output and load valid JSON
        output_dict = gen_pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)
        output = output_dict[0]["generated_text"][len(prompt):]
        json_result = json.loads(output)

        # Return valid JSON object according to the schema DeepEval supplied
        return schema(**json_result)

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Gemma-2B Unsloth"


----------

------------

In [16]:
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

test_cases = [
    LLMTestCase(
        input="I'm feeling really overwhelmed with my workload. How can I manage my stress?",
        actual_output="""Hey there, I understand that feeling overwhelmed can be really tough. It's important to take steps to manage stress and prioritize your well-being.

**Here are some things you can try:**

* **Prioritize self-care:** Make time for activities that bring you joy and relaxation, like reading, taking a walk, or listening to music.
* **Set realistic goals:** Break down your tasks into smaller, manageable steps.
* **Delegate tasks:** If possible, delegate some of your responsibilities to others.
* **Take breaks:** Step away from your work for a few minutes every hour or so.""",
        expected_output="It's important to prioritize and break down your tasks into manageable chunks. Let's work on some stress-reduction techniques, like mindfulness and deep breathing exercises.",
        retrieval_context=["Stress management involves breaking tasks into manageable steps, prioritizing, practicing relaxation techniques, and seeking support when necessary."]
    ),
    LLMTestCase(
        input="I'm struggling with my body image. I just don't feel good about myself.",
        actual_output="""Hey there, I'm so sorry to hear that you're feeling that way. It's completely normal to struggle with body image issues, and it's important to remember that you're not alone in this. Many people experience body image issues, and it's a sign of self-love and acceptance.

It's important to take things one step at a time when working on body image issues. Start by focusing on self-compassion and kindness. Treat yourself with the same compassion and understanding you would offer to a friend going through a tough time.

Remember, body image is just one aspect of your""",
        expected_output="It's important to recognize that our bodies are unique and it's okay to have imperfections. Consider practicing self-care habits such as exercise, healthy eating, and positive self-talk. Let's work together to improve your self-esteem and body image.",
        retrieval_context=["Improving body image involves developing self-acceptance, positive self-talk, self-care practices like exercise, and understanding body diversity."]
    ),
    LLMTestCase(
        input="I'm having trouble concentrating and staying focused. What can I do to improve my cognitive function?",
        actual_output="""Hey there, and thank you for reaching out for help. It's completely understandable to feel overwhelmed by the constant stream of information and distractions around us. Let's work together to identify some strategies that might help you regain your focus and concentration.

**Here are some things you can try:**

* **Prioritize sleep:** Getting enough sleep is crucial for optimal cognitive function. Aim for 7-8 hours of sleep each night.
* **Stay hydrated:** Dehydration can significantly impact focus and concentration. Make sure to drink plenty of water throughout the day.
* **Take breaks:** Regular breaks can help you recharge and""",
        expected_output="There are several strategies you can try to boost your cognitive function. For example, make sure you're getting enough sleep and exercise, as these are important for brain health. It's also helpful to challenge your brain regularly, by engaging in activities like puzzles, reading, or learning a new skill. And make sure you're taking care of your mental health, by managing stress and seeking treatment for any underlying conditions like ADHD or depression.",
        retrieval_context=["Improving cognitive function involves physical activity, mindfulness meditation, brain exercises, proper sleep, and stress management."]
    ),
    LLMTestCase(
        input="I think I might have depression. What should I do?",
        actual_output="""Hey there, and thank you for reaching out for help. It's completely understandable to feel overwhelmed and uncertain about what to do when you think you might be experiencing depression.

I understand that depression can feel overwhelming and scary, but it's important to remember that you're not alone. Many people experience similar feelings, and there are effective treatments available.

Here are some things you can do to take care of yourself and start the healing process:

* **Reach out for professional help:** Consider talking to a therapist or counselor who specializes in depression. They can provide a safe and supportive space for you to explore your feelings and.""",
        expected_output="It's great that you're recognizing your symptoms and seeking help. I can provide you with a referral to a mental health professional who can diagnose and treat depression.",
        retrieval_context=["Seeking professional help for depression involves seeing a therapist or psychiatrist for diagnosis and treatment."]
    ),
    LLMTestCase(
        input="I'm having trouble concentrating and I keep forgetting things.",
        actual_output="""Hey there, I understand that you're feeling overwhelmed and forgetful. It's important to acknowledge these feelings and let yourself feel them without judgment. Remember, everyone experiences moments of forgetfulness, and it's nothing to be ashamed of.

Let's work together to figure out what might be causing this feeling and find some strategies to help you manage it. We can explore different techniques like mindfulness exercises, time management strategies, and relaxation techniques.

I'm here to listen and support you on your journey to better focus and memory. Remember, taking care of your mental health is just as important as taking care of your""",
        expected_output="Let's talk about what might be causing those symptoms. Have you been feeling stressed or overwhelmed lately?",
        retrieval_context=["Difficulty concentrating can be linked to stress, anxiety, ADHD, and other factors that affect focus and memory."]
    ),
    LLMTestCase(
        input="I'm having trouble with my self-esteem and confidence.",
        actual_output="""Hey there, and thank you for reaching out for support. It's completely understandable to feel insecure and uncertain about yourself sometimes. It's important to remember that self-esteem is not a single thing, but rather a collection of various factors that contribute to how we perceive ourselves.

Let's work together to explore what's contributing to these feelings and help you build a more positive self-image. Together, we can identify any limiting beliefs or negative patterns you may have developed, and then work on replacing them with more empowering and realistic ones.

Remember, healing is a journey, and it takes time and effort. But""",
        expected_output="Low self-esteem can be a difficult issue to overcome, but it's possible with the right tools and support. Let's talk about any negative self-talk or beliefs you may have and explore different strategies to help improve your self-esteem and confidence.",
        retrieval_context=["Improving self-esteem involves challenging negative self-talk, developing positive beliefs, and seeking support through therapy or self-care."]
    ),
    LLMTestCase(
        input="I'm having trouble with my work-life balance and I feel like I'm constantly stressed.",
        actual_output="""Hey there, I understand that managing work-life balance can be challenging, especially when stress becomes a constant presence. It's important to prioritize self-care and prioritize your well-being.

**Here are some things you can try to improve your work-life balance:**

* **Set boundaries:** Establish clear work hours and stick to them as much as possible.
* **Delegate tasks:** If possible, delegate some of your responsibilities to others.
* **Take breaks:** Schedule regular breaks throughout the day to recharge and clear your head.
* **Learn to say no:** Don't be afraid to decline additional""",
        expected_output="Work-life balance can be a difficult issue to navigate, but it's important to prioritize self-care and set boundaries to prevent burnout. Let's work on some strategies to help you manage your stress, such as practicing relaxation techniques, delegating tasks, and scheduling time for self-care activities.",
        retrieval_context=["Improving work-life balance involves self-care, boundary setting, stress management techniques, and prioritizing personal well-being."]
    ),
    LLMTestCase(
        input="I'm feeling really angry all the time. What can I do to manage my anger?",
        actual_output="""Hey there, I understand that you're feeling angry a lot. It's completely normal to feel angry sometimes, but it's important to find healthy ways to manage it.

**Here are some things you can try:**

* **Take a few deep breaths.** This can help to calm your body and mind.
* **Identify what's making you angry.** Once you know what's triggering your anger, you can start to avoid those situations or develop coping mechanisms for dealing with them.
* **Talk to a trusted friend, family member, or therapist.** They can offer support and guidance.
* **""",
        expected_output="One technique that may be helpful is called 'counting to ten.' When you feel yourself getting angry, take a deep breath and count to ten before reacting. This can give you time to calm down and respond in a more rational and productive way. Additionally, practicing relaxation techniques like deep breathing or meditation can also be effective in managing anger.",
        retrieval_context=["Managing anger involves identifying triggers, using coping mechanisms like deep breathing and mindfulness, and developing personalized strategies."]
    ),
    LLMTestCase(
        input="I'm feeling really stressed at work. I don't know how to manage my workload and it's starting to affect my health.",
        actual_output="""Hey there, I understand that feeling. Managing a demanding workload can be quite challenging, especially when it comes to maintaining your physical and mental well-being. It's important to acknowledge the stress you're experiencing and prioritize your self-care.

**Here are some things you can try to manage your workload and improve your overall well-being:**

* **Set realistic goals:** Break down your tasks into smaller, manageable steps. This can make them seem less overwhelming and easier to handle.
* **Prioritize tasks:** Identify the most important tasks and focus on completing them first.
* **Delegate tasks:** If possible,""",
        expected_output="Work stress can be really challenging, but it's possible to find a balance. Let's explore some time management strategies and ways to prioritize your tasks. We can also work on stress management techniques to help you cope with the pressure.",
        retrieval_context=["Managing work stress involves time management, identifying stressors, prioritizing tasks, and employing stress-reduction techniques."]
    ),
    LLMTestCase(
        input="I tried telling my husband I was depressed, and he ignored me. He said 'you're always sad or depressed.' And he picked up his phone and ignored me. I said, 'Please don't exaggerate, that isn't true.' And he said, 'Whatever babe. You just want to be sad.' How can I get through to him so he will take me seriously?",
        actual_output="""Hey there, I understand that you're feeling hurt and disrespected by your husband. It's important to remember that your feelings are valid, and that you deserve to be treated with respect.

It's also important to communicate your needs and feelings to your partner in a non-judgmental and respectful way. Try to express yourself in a calm and non-accusatory tone, and be specific about what you're feeling and how it's affecting you.

Let your husband know that you love and appreciate him, and that you're there for him no matter what. Be patient and understanding, and let him""",
        expected_output="Oh dear. From what you write, your husband doesn't have any interest to know about you and your sad feelings. Stay clear in understanding that you already did your part by asking him to listen and care about the way you feel. In this example, he told you he doesn't care. It's up to him to open his mind and heart to you. Depending on how motivated you are to continue asking him to hear you, try until you feel satisfied that you've given him enough chances to know who you are. At a certain number of tries at this, you'll either have success or not. Just don't try forever because doing so will increase your depressed feelings.",
        retrieval_context=["Effective communication in relationships involves understanding, expressing feelings, and recognizing when further attempts at communication may be unproductive."]
    ),
    LLMTestCase(
        input="I'm struggling with low self-esteem and don't know how to improve it.",
        actual_output="""Hey there, and thank you for reaching out for support. It's completely understandable to feel low in self-esteem sometimes, especially if you're facing challenges or feeling overwhelmed. It's important to remember that self-esteem is not a single thing, but rather a collection of various thoughts, feelings, and beliefs about yourself.

Let's work together to explore what's contributing to your low self-esteem and come up with a plan to help you build it back up.

Here are some things we can explore:

* **Past experiences:** What experiences have shaped your self-esteem?
* **Current""",
        expected_output="Self-esteem is something that can be improved with time and effort. Let's talk about what specifically is causing you to feel this way and come up with some strategies to help you build your confidence.",
        retrieval_context=["Improving self-esteem involves identifying root causes, cognitive-behavioral strategies, setting achievable goals, and practicing self-compassion."]
    ),
    LLMTestCase(
        input="I'm feeling really overwhelmed with my workload. How can I manage my stress?",
        actual_output="""Hey there, I understand that feeling overwhelmed can be really tough. It's important to take steps to manage stress and prioritize your well-being.

**Here are some things you can try:**

* **Prioritize self-care:** Make time for activities that bring you joy and relaxation, like reading, taking a walk, or listening to music.
* **Set realistic goals:** Break down your tasks into smaller, manageable steps.
* **Delegate tasks:** If possible, delegate some of your responsibilities to others.
* **Take breaks:** Step away from your work for a few minutes every hour or so.""",
        expected_output="It's important to prioritize and break down your tasks into manageable chunks. Let's work on some stress-reduction techniques, like mindfulness and deep breathing exercises.",
        retrieval_context=["The user expressed feeling overwhelmed due to workload and seeking stress management tips."]
    ),
    LLMTestCase(
        input="I'm struggling with my body image. I just don't feel good about myself.",
        actual_output="""Hey there, I'm so sorry to hear that you're feeling that way. It's completely normal to struggle with body image issues, and it's important to remember that you're not alone in this. Many people experience body image issues, and it's a sign of self-love and acceptance.

It's important to take things one step at a time when working on body image issues. Start by focusing on self-compassion and kindness. Treat yourself with the same compassion and understanding you would offer to a friend going through a tough time.

Remember, body image is just one aspect of your""",
        expected_output="It's important to recognize that our bodies are unique and it's okay to have imperfections. Consider practicing self-care habits such as exercise, healthy eating, and positive self-talk. Let's work together to improve your self-esteem and body image.",
        retrieval_context=["The user shared concerns about body image and self-esteem."]
    ),
    LLMTestCase(
        input="I'm having trouble concentrating and staying focused. What can I do to improve my cognitive function?",
        actual_output="""Hey there, and thank you for reaching out for help. It's completely understandable to feel overwhelmed by the constant stream of information and distractions around us. Let's work together to identify some strategies that might help you regain your focus and concentration.

**Here are some things you can try:**

* **Prioritize sleep:** Getting enough sleep is crucial for optimal cognitive function. Aim for 7-8 hours of sleep each night.
* **Stay hydrated:** Dehydration can significantly impact focus and concentration. Make sure to drink plenty of water throughout the day.
* **Take breaks:** Regular breaks can help you recharge and""",
        expected_output="There are several strategies you can try to boost your cognitive function. For example, make sure you're getting enough sleep and exercise, as these are important for brain health. It's also helpful to challenge your brain regularly, by engaging in activities like puzzles, reading, or learning a new skill. And make sure you're taking care of your mental health, by managing stress and seeking treatment for any underlying conditions like ADHD or depression.",
        retrieval_context=["The user expressed difficulty in concentrating and staying focused, asking for cognitive improvement strategies."]
    ),
    LLMTestCase(
        input="I think I might have depression. What should I do?",
        actual_output="""Hey there, and thank you for reaching out for help. It's completely understandable to feel overwhelmed and uncertain about what to do when you think you might be experiencing depression.

I understand that depression can feel overwhelming and scary, but it's important to remember that you're not alone. Many people experience similar feelings, and there are effective treatments available.

Here are some things you can do to take care of yourself and start the healing process:

* **Reach out for professional help:** Consider talking to a therapist or counselor who specializes in depression. They can provide a safe and supportive space for you to explore your feelings and""",
        expected_output="It's great that you're recognizing your symptoms and seeking help. I can provide you with a referral to a mental health professional who can diagnose and treat depression.",
        retrieval_context=["The user is concerned about potential depression and seeking guidance on next steps."]
    ),
    LLMTestCase(
        input="I'm having trouble concentrating and I keep forgetting things.",
        actual_output="""Hey there, I understand that you're feeling overwhelmed and forgetful. It's important to acknowledge these feelings and let yourself feel them without judgment. Remember, everyone experiences moments of forgetfulness, and it's nothing to be ashamed of.

Let's work together to figure out what might be causing this feeling and find some strategies to help you manage it. We can explore different techniques like mindfulness exercises, time management strategies, and relaxation techniques.

I'm here to listen and support you on your journey to better focus and memory. Remember, taking care of your mental health is just as important as taking care of your""",
        expected_output="Let's talk about what might be causing those symptoms. Have you been feeling stressed or overwhelmed lately?",
        retrieval_context=["The user expressed difficulty concentrating and forgetfulness, asking for possible causes and solutions."]
    ),
    LLMTestCase(
        input="I'm having trouble with my self-esteem and confidence.",
        actual_output="""Hey there, and thank you for reaching out for support. It's completely understandable to feel insecure and uncertain about yourself sometimes. It's important to remember that self-esteem is not a single thing, but rather a collection of various factors that contribute to how we perceive ourselves.

Let's work together to explore what's contributing to these feelings and help you build a more positive self-image. Together, we can identify any limiting beliefs or negative patterns you may have developed, and then work on replacing them with more empowering and realistic ones.

Remember, healing is a journey, and it takes time and effort. But""",
        expected_output="Low self-esteem can be a difficult issue to overcome, but it's possible with the right tools and support. Let's talk about any negative self-talk or beliefs you may have and explore different strategies to help improve your self-esteem and confidence.",
        retrieval_context=["The user expressed concerns about low self-esteem and confidence, asking for ways to improve."]
    ),
    LLMTestCase(
        input="I'm having trouble with my work-life balance and I feel like I'm constantly stressed.",
        actual_output="""Hey there, I understand that managing work-life balance can be challenging, especially when stress becomes a constant presence. It's important to prioritize self-care and prioritize your well-being.

**Here are some things you can try to improve your work-life balance:**

* **Set boundaries:** Establish clear work hours and stick to them as much as possible.
* **Delegate tasks:** If possible, delegate some of your responsibilities to others.
* **Take breaks:** Schedule regular breaks throughout the day to recharge and clear your head.
* **Learn to say no:** Don't be afraid to decline additional""",
        expected_output="Work-life balance can be a difficult issue to navigate, but it's important to prioritize self-care and set boundaries to prevent burnout. Let's work on some strategies to help you manage your stress, such as practicing relaxation techniques, delegating tasks, and scheduling time for self-care activities.",
        retrieval_context=["The user mentioned struggles with work-life balance and stress, seeking strategies for improvement."]
    ),
    LLMTestCase(
        input="I'm feeling really angry all the time. What can I do to manage my anger?",
        actual_output="""Hey there, I understand that you're feeling angry a lot. It's completely normal to feel angry sometimes, but it's important to find healthy ways to manage it.

**Here are some things you can try:**

* **Take a few deep breaths.** This can help to calm your body and mind.
* **Identify what's making you angry.** Once you know what's triggering your anger, you can start to avoid those situations or develop coping mechanisms for dealing with them.
* **Talk to a trusted friend, family member, or therapist.** They can offer support and guidance.
* **""",
        expected_output="One technique that may be helpful is called 'counting to ten.' When you feel yourself getting angry, take a deep breath and count to ten before reacting. This can give you time to calm down and respond in a more rational and productive way. Additionally, practicing relaxation techniques like deep breathing or meditation can also be effective in managing anger.",
        retrieval_context=["The user expressed constant anger and sought methods for anger management."]
    ),
    LLMTestCase(
        input="I'm feeling really stressed at work. I don't know how to manage my workload and it's starting to affect my health.",
        actual_output="""Hey there, I understand that feeling. Managing a demanding workload can be quite challenging, especially when it comes to maintaining your physical and mental well-being. It's important to acknowledge the stress you're experiencing and prioritize your self-care.

**Here are some things you can try to manage your workload and improve your overall well-being:**

* **Set realistic goals:** Break down your tasks into smaller, manageable steps. This can make them seem less overwhelming and easier to handle.
* **Prioritize tasks:** Identify the most important tasks and focus on completing them first.
* **Delegate tasks:** If possible,""",
        expected_output="Work stress can be really challenging, but it's possible to find a balance. Let's explore some time management strategies and ways to prioritize your tasks. We can also work on stress management techniques to help you cope with the pressure.",
        retrieval_context=["The user is facing work-related stress and is asking for advice on workload management."]
    ),
    LLMTestCase(
        input="I tried telling my husband I was depressed, and he ignored me. He said 'you're always sad or depressed.' And he picked up his phone and ignored me. I said, 'Please don't exaggerate, that isn't true.' And he said, 'Whatever babe. You just want to be sad.' How can I get through to him so he will take me seriously?",
        actual_output="""Hey there, I understand that you're feeling hurt and disrespected by your husband. It's important to remember that your feelings are valid, and that you deserve to be treated with respect.

It's also important to communicate your needs and feelings to your partner in a non-judgmental and respectful way. Try to express yourself in a calm and non-accusatory tone, and be specific about what you're feeling and how it's affecting you.

Let your husband know that you love and appreciate him, and that you're there for him no matter what. Be patient and understanding, and let him""",
        expected_output="Oh dear. From what you write, your husband doesn't have any interest to know about you and your sad feelings. Stay clear in understanding that you already did your part by asking him to listen and care about the way you feel. In this example, he told you he doesn't care. It's up to him to open his mind and heart to you. Depending on how motivated you are to continue asking him to hear you, try until you feel satisfied that you've given him enough chances to know who you are. At a certain number of tries at this, you'll either have success or not. Just don't try forever because doing so will increase your depressed feelings.",
        retrieval_context=["The user shared a difficult conversation with their husband, seeking advice on how to be taken seriously regarding depression."]
    ),
]

In [13]:
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric,
    BiasMetric,
    ToxicityMetric,
    GEval
)
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval import evaluate

custom_llm = CustomModel()

answer_relevancy = AnswerRelevancyMetric(threshold=0.6, model=custom_llm)


# test_cases
result = evaluate(
    test_cases=test_cases,
    metrics=[answer_relevancy],
    print_results=True,  # Change to True to see detailed metric results
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Gemma-2B Unsloth, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 21 test case(s) in parallel: |          |  0% (0/21) [Time Taken: 00:00, ?test case/s]Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |▍         |  5% (1/21) [Time Taken: 02:06, 126.75s/test case]Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |▉         | 10% (2/21) [Time Taken: 03:05, 86.70s/test case] Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s)



Metrics Summary

  - ✅ Answer Relevancy (score: 0.6666666666666666, threshold: 0.6, strict: False, evaluation model: Gemma-2B Unsloth, reason: The score is 0.67 because the input mentions feeling overwhelmed with stress, but the actual output does not address or provide any solutions for managing stress., error: None)

For test case:

  - input: I'm feeling really overwhelmed with my workload. How can I manage my stress?
  - actual output: Hey there, I understand that feeling overwhelmed can be really tough. It's important to take steps to manage stress and prioritize your well-being.

**Here are some things you can try:**

* **Prioritize self-care:** Make time for activities that bring you joy and relaxation, like reading, taking a walk, or listening to music.
* **Set realistic goals:** Break down your tasks into smaller, manageable steps.
* **Delegate tasks:** If possible, delegate some of your responsibilities to others.
* **Take breaks:** Step away from your work for a few minute

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [15]:
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric,
    BiasMetric,
    ToxicityMetric,
    GEval
)
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval import evaluate

# custom_llm = CustomModel()

bias = BiasMetric(threshold=0.7, model=custom_llm)

# test_cases
result = evaluate(
    test_cases=test_cases,
    metrics=[bias],
    print_results=True,  # Change to True to see detailed metric results
)

✨ You're running DeepEval's latest Bias Metric! (using Gemma-2B Unsloth, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 21 test case(s) in parallel: |          |  0% (0/21) [Time Taken: 00:00, ?test case/s]Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |▍         |  5% (1/21) [Time Taken: 01:54, 114.03s/test case]Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |▉         | 10% (2/21) [Time Taken: 02:43, 75.98s/test case] Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |█▍        | 14% (3/21) [Time Taken: 03:40, 67.24s/test case]Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |█▉        | 19% (4/21) [Time Taken: 04:30, 60.70s/test case]Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |██▍   



Metrics Summary

  - ✅ Bias (score: 0.3333333333333333, threshold: 0.7, strict: False, evaluation model: Gemma-2B Unsloth, reason: This opinion promotes unrealistic self-expectations, setting a poor standard for success and potentially leading to disappointment or burnout., error: None)

For test case:

  - input: I'm feeling really overwhelmed with my workload. How can I manage my stress?
  - actual output: Hey there, I understand that feeling overwhelmed can be really tough. It's important to take steps to manage stress and prioritize your well-being.

**Here are some things you can try:**

* **Prioritize self-care:** Make time for activities that bring you joy and relaxation, like reading, taking a walk, or listening to music.
* **Set realistic goals:** Break down your tasks into smaller, manageable steps.
* **Delegate tasks:** If possible, delegate some of your responsibilities to others.
* **Take breaks:** Step away from your work for a few minutes every hour or so.
  - expected

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [17]:
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric,
    BiasMetric,
    ToxicityMetric,
    GEval
)
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval import evaluate

custom_llm = CustomModel()

faithfulness = FaithfulnessMetric(threshold=0.6, model=custom_llm)


# test_cases
result = evaluate(
    test_cases=test_cases,
    metrics=[faithfulness],
    print_results=True,  # Change to True to see detailed metric results
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

...

✨ You're running DeepEval's latest Faithfulness Metric! (using Gemma-2B Unsloth, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 21 test case(s) in parallel: |          |  0% (0/21) [Time Taken: 00:00, ?test case/s]Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) in parallel: |▍         |  5% (1/21) [Time Taken: 04:38, 278.40s/test case]Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Evaluating 21 test case(s) i



Metrics Summary

  - ❌ Faithfulness (score: 0.3333333333333333, threshold: 0.6, strict: False, evaluation model: Gemma-2B Unsloth, reason: The score is 0.33 because the contradictions indicate that the actual output does not align with the information presented in the retrieval context., error: None)

For test case:

  - input: I'm feeling really overwhelmed with my workload. How can I manage my stress?
  - actual output: Hey there, I understand that feeling overwhelmed can be really tough. It's important to take steps to manage stress and prioritize your well-being.

**Here are some things you can try:**

* **Prioritize self-care:** Make time for activities that bring you joy and relaxation, like reading, taking a walk, or listening to music.
* **Set realistic goals:** Break down your tasks into smaller, manageable steps.
* **Delegate tasks:** If possible, delegate some of your responsibilities to others.
* **Take breaks:** Step away from your work for a few minutes every hour or so

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.